In [ ]:
HOMEWORK 1

In [1]:
#Install pysqlite3 for python and import pandas to use later
#!pip install pysqlite3
from sqlite3 import dbapi2 as sqlite3
print(sqlite3.sqlite_version)
import pandas as pd
from IPython.display import display, HTML

3.45.3


In [3]:
dbname = "homework1-1.db"

def printSqlResults(cursor, tblName):
  try:
    df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
    display(HTML("<b><font color=Green> " + tblName + "</font></b>" + df.to_html(index=False)))
  except:
    pass

def runSql(caption, query):
  conn = sqlite3.connect(dbname) # Connect to the database
  cursor = conn.cursor() # Create a cursor (think: it's like a "pointer")
  cursor.execute(query) # Execute the query
  printSqlResults(cursor, caption) # Print the results
  conn.close()

def runSql_withCommit(caption, query):
  conn = sqlite3.connect(dbname) # Connect to the database
  cursor = conn.cursor() # Create a cursor (think: it's like a "pointer")
  cursor.execute(query) # Execute the query
  printSqlResults(cursor, caption) # Print the results
  conn.commit()
  conn.close()

def runStepByStepSql(query, fromline):
  lines = query.strip().split('\n')
  for lineidx in range(fromline, len(lines)):
    partial_query = '\n'.join(lines[:lineidx])
    caption = 'Query till line:' +  partial_query
    runSql(caption, partial_query + ';')

In [5]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("homework1-1.db")
cursor = conn.cursor()

# Drop tables if they exist (to prevent duplication)
cursor.executescript("""
DROP TABLE IF EXISTS EMPLOYEE;
DROP TABLE IF EXISTS DEPARTMENT;
DROP TABLE IF EXISTS WORKS_ON;
DROP TABLE IF EXISTS PROJECT;
DROP TABLE IF EXISTS DEPENDENT;
DROP TABLE IF EXISTS DEPT_LOCATIONS;
""")

# Create tables
cursor.executescript("""
CREATE TABLE EMPLOYEE (
    Fname TEXT,
    Minit TEXT,
    Lname TEXT,
    Ssn TEXT PRIMARY KEY,
    Bdate TEXT,
    Address TEXT,
    Sex TEXT,
    Salary INTEGER,
    Super_ssn TEXT,
    Dno INTEGER
);

CREATE TABLE DEPARTMENT (
    Dname TEXT,
    Dnumber INTEGER PRIMARY KEY,
    Mgr_ssn TEXT,
    Mgr_start_date TEXT
);

CREATE TABLE WORKS_ON (
    Essn TEXT,
    Pno INTEGER,
    Hours REAL,
    FOREIGN KEY (Essn) REFERENCES EMPLOYEE(Ssn)
);

CREATE TABLE PROJECT (
    Pname TEXT,
    Pnumber INTEGER PRIMARY KEY,
    Plocation TEXT,
    Dnum INTEGER,
    FOREIGN KEY (Dnum) REFERENCES DEPARTMENT(Dnumber)
);

CREATE TABLE DEPENDENT (
    Essn TEXT,
    Dependent_name TEXT,
    Sex TEXT,
    Bdate TEXT,
    Relationship TEXT,
    FOREIGN KEY (Essn) REFERENCES EMPLOYEE(Ssn)
);

CREATE TABLE DEPT_LOCATIONS (
    Dnumber INTEGER,
    Dlocation TEXT,
    FOREIGN KEY (Dnumber) REFERENCES DEPARTMENT(Dnumber)
);
""")

conn.commit()

In [7]:
# Insert Employee Data
cursor.executemany("""
INSERT INTO EMPLOYEE VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
""", [
    ("John", "B", "Smith", "123456789", "1965-01-09", "731 Fondren, Houston, TX", "M", 30000, "333445555", 5),
    ("Franklin", "T", "Wong", "333445555", "1955-12-08", "638 Voss, Houston, TX", "M", 40000, "888665555", 5),
    ("Alicia", "J", "Zelaya", "999887777", "1968-01-19", "3321 Castle, Spring, TX", "F", 25000, "987654321", 4),
    ("Jennifer", "S", "Wallace", "987654321", "1941-06-20", "291 Berry, Bellaire, TX", "F", 43000, "888665555", 4),
    ("Ramesh", "K", "Narayan", "666884444", "1962-09-15", "975 Fire Oak, Humble, TX", "M", 38000, "333445555", 5),
    ("Joyce", "A", "English", "453454353", "1972-07-31", "5631 Rice, Houston, TX", "F", 25000, "333445555", 5),
    ("Ahmad", "V", "Jabbar", "987987987", "1969-03-29", "980 Dallas, Houston, TX", "M", 25000, "987654321", 4),
    ("James", "E", "Borg", "888665555", "1937-11-10", "450 Stone, Houston, TX", "M", 55000, None, 1)
])

# Insert Department Data
cursor.executemany("""
INSERT INTO DEPARTMENT VALUES (?, ?, ?, ?);
""", [
    ("Research", 5, "333445555", "1988-05-22"),
    ("Administration", 4, "987654321", "1995-01-01"),
    ("Headquarters", 1, "888665555", "1981-06-19")
])

# Insert Works_On Data
cursor.executemany("""
INSERT INTO WORKS_ON VALUES (?, ?, ?);
""", [
    ("123456789", 1, 32.5),
    ("123456789", 2, 7.5),
    ("666884444", 3, 40.0),
    ("453454353", 1, 20.0),
    ("453454353", 2, 20.0),
    ("333445555", 10, 10.0),
    ("333445555", 20, 10.0),
    ("333445555", 30, 10.0),
    ("999887777", 30, 10.0),
    ("987987987", 10, 35.0),
    ("987654321", 30, 20.0),
    ("987654321", 20, 15.0),
    ("888665555", 20, None)
])

# Insert Project Data
cursor.executemany("""
INSERT INTO PROJECT VALUES (?, ?, ?, ?);
""", [
    ("ProductX", 1, "Bellaire", 5),
    ("ProductY", 2, "Sugarland", 5),
    ("ProductZ", 3, "Houston", 5),
    ("Computerization", 10, "Stafford", 4),
    ("Reorganization", 20, "Houston", 1),
    ("Newbenefits", 30, "Stafford", 4)
])

# Insert Dependent Data
cursor.executemany("""
INSERT INTO DEPENDENT VALUES (?, ?, ?, ?, ?);
""", [
    ("333445555", "Alice", "F", "1986-04-05", "Daughter"),
    ("333445555", "Theodore", "M", "1983-10-25", "Son"),
    ("333445555", "Joy", "F", "1958-05-03", "Spouse"),
    ("987654321", "Abner", "M", "1942-02-28", "Spouse"),
    ("123456789", "Michael", "M", "1988-01-04", "Son"),
    ("123456789", "Alice", "F", "1988-12-30", "Daughter"),
    ("123456789", "Elizabeth", "F", "1967-05-05", "Spouse")
])

# Insert Department Locations
cursor.executemany("""
INSERT INTO DEPT_LOCATIONS VALUES (?, ?);
""", [
    (1, "Houston"),
    (4, "Stafford"),
    (5, "Bellaire"),
    (5, "Sugarland"),
    (5, "Houston")
])

# Commit changes and close the connection
conn.commit()
conn.close()

print("Database successfully created and populated!")
runSql('EMPLOYEE', "select * from EMPLOYEE;")
runSql('DEPARTMENT', "select * from DEPARTMENT;")
runSql('WORKS_ON', "select * from WORKS_ON;")
runSql('PROJECT', "select * from PROJECT;")
runSql('DEPENDENT', "select * from DEPENDENT;")
runSql('DEPT_LOCATIONS', "select * from DEPT_LOCATIONS;")

Database successfully created and populated!


Fname,Minit,Lname,Ssn,Bdate,Address,Sex,Salary,Super_ssn,Dno
John,B,Smith,123456789,1965-01-09,"731 Fondren, Houston, TX",M,30000,333445555,5
Franklin,T,Wong,333445555,1955-12-08,"638 Voss, Houston, TX",M,40000,888665555,5
Alicia,J,Zelaya,999887777,1968-01-19,"3321 Castle, Spring, TX",F,25000,987654321,4
Jennifer,S,Wallace,987654321,1941-06-20,"291 Berry, Bellaire, TX",F,43000,888665555,4
Ramesh,K,Narayan,666884444,1962-09-15,"975 Fire Oak, Humble, TX",M,38000,333445555,5
Joyce,A,English,453454353,1972-07-31,"5631 Rice, Houston, TX",F,25000,333445555,5
Ahmad,V,Jabbar,987987987,1969-03-29,"980 Dallas, Houston, TX",M,25000,987654321,4
James,E,Borg,888665555,1937-11-10,"450 Stone, Houston, TX",M,55000,None,1


Dname,Dnumber,Mgr_ssn,Mgr_start_date
Headquarters,1,888665555,1981-06-19
Administration,4,987654321,1995-01-01
Research,5,333445555,1988-05-22


Essn,Pno,Hours
123456789,1,32.5
123456789,2,7.5
666884444,3,40.0
453454353,1,20.0
453454353,2,20.0
333445555,10,10.0
333445555,20,10.0
333445555,30,10.0
999887777,30,10.0
987987987,10,35.0


Pname,Pnumber,Plocation,Dnum
ProductX,1,Bellaire,5
ProductY,2,Sugarland,5
ProductZ,3,Houston,5
Computerization,10,Stafford,4
Reorganization,20,Houston,1
Newbenefits,30,Stafford,4


Essn,Dependent_name,Sex,Bdate,Relationship
333445555,Alice,F,1986-04-05,Daughter
333445555,Theodore,M,1983-10-25,Son
333445555,Joy,F,1958-05-03,Spouse
987654321,Abner,M,1942-02-28,Spouse
123456789,Michael,M,1988-01-04,Son
123456789,Alice,F,1988-12-30,Daughter
123456789,Elizabeth,F,1967-05-05,Spouse


Dnumber,Dlocation
1,Houston
4,Stafford
5,Bellaire
5,Sugarland
5,Houston
